## Notebook to create LDA of 40 topics with speeches

In [1]:
import pandas as pd
import os
from operator import itemgetter
import gensim
import re
from tqdm import tqdm
from os import listdir
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
from collections import Counter

In [6]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [4]:
os.chdir('./../../../../../data/02-outcomes/01-policy_positioning/01-text_analysis/wordfish/01_lda/')

## Load Clean Data

In [33]:
def remove_infrequent_words(data):
    remove = list(pd.read_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/wordfish/01_lda/remove_words.csv').remove_words)
    extra_words = ['diputar', 'ley', 
              'articular', 'publicar', 'federal']
    remove  = remove + extra_words
    remove_list = '|'.join(remove)
    regex = re.compile(r'\b('+remove_list+r')\b', flags=re.IGNORECASE)
    
    speech_clean = []
    for x in tqdm(data.clean_speech):
        clean = regex.sub('', x)
        clean = re.sub('[ ]+', ' ', clean)
        speech_clean.append(clean)
    return(speech_clean)

In [15]:
def load_data():
    
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/01-clean_data/disaaggregated_data/'
    data = []
    for i in listdir(path):
        if any(re.findall('json$', i)):
            data.append(pd.read_json(path + i)[['id_speech', 'clean_speech']])
           
    data = pd.concat(data)
    data = data[data['clean_speech'] != '']
    return(data)

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


In [35]:
data = load_data()

In [40]:
data_clean = remove_infrequent_words(data)

100%|██████████| 40414/40414 [4:56:45<00:00,  2.27it/s]   


In [41]:
data['data_clean'] = data_clean

In [43]:
data.to_excel('/Users/cblanesg/Desktop/clean_speaches.xlsx')

In [51]:
import json

In [52]:
with open('/Users/cblanesg/Desktop/clean_speaches.json', 'w') as f:
    json.dump(data.to_dict(), f)

In [13]:
def lda_model_function(num_topics, corpus, id2word):
    print('training model')
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics)
    
    return(lda_model)

In [12]:
def load_corpus(data):
        ## prepare inputs
    print('loding data')
    #data = load_data()
    data = data.data_clean.values.tolist()
    data_words = list(sent_to_words(data))
    
    # Create Dictionary
    print('preparing input')
    id2word = corpora.Dictionary(data_words)

    # Create Corpus
    texts = data_words

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    return(data_words, corpus, id2word)

In [11]:
def obtain_topics_docs(lda_model, corpus):
    
    all_topics = []
    for i in tqdm(corpus):
        main_topic = max(lda_model[i],key=itemgetter(1))[0]  
        all_topics.append(main_topic + 1)
    return(all_topics)
    

In [10]:
def obtain_visualization(num_topics, lda_model, corpus, id2word):
    LDAvis_data_filepath = os.path.join('./lda_visualization/lda_prepared_'+str(num_topics))
    
    if 1 == 1:
        LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)

    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)

    pyLDAvis.save_html(LDAvis_prepared, './lda_visualization/lda_prepared_'+ str(num_topics) +'.html')
    LDAvis_prepared

## LDA

In [7]:
data = pd.read_excel('./clean_speaches_topics.xlsx')

<ipython-input-7-a551e6f736fd>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  data = pd.read_excel('./clean_speaches_topics.xlsx')


In [16]:
data_words, corpus20, id2word20 = load_corpus(data)

loding data
preparing input


In [21]:
lda_model15k = lda_model_function(15, corpus20, id2word20)

training model


## Analysis LDA

In [18]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [19]:
pyLDAvis.enable_notebook()

In [22]:
obtain_visualization(15, lda_model15k, corpus20, id2word20)

## Check best number of topics

In [37]:
import json

In [138]:
data_text = load_data()

In [28]:
topics20k = obtain_topics_docs(lda_model30k, corpus20)

100%|██████████| 40414/40414 [00:35<00:00, 1149.99it/s]


In [29]:
data['topics20k'] = topics20k

In [35]:
data = data.drop(['Unnamed: 0'], axis = 1)

In [38]:
with open('/Users/cblanesg/Desktop/clean_speaches_topics.json', 'w') as f:
    json.dump(data.to_dict(), f)

In [39]:
data.to_excel('/Users/cblanesg/Desktop/clean_speaches_topics.xlsx')

## distribution of percentage of topics

In [50]:
def obtain_topics_docs_percentage(lda_model, corpus):
    
    all_topics = []
    for i in tqdm(corpus):
        main_topic = max(lda_model[i],key=itemgetter(1))[1]  
        all_topics.append(main_topic)
    return(all_topics)

In [52]:
percentage_topics20k = obtain_topics_docs_percentage(lda_model30k, corpus20)
percentage_topics15k = obtain_topics_docs_percentage(lda_model15k, corpus20)

100%|██████████| 40414/40414 [00:31<00:00, 1290.02it/s]


In [53]:
data['percentage_topics20k'] = percentage_topics20k
data['percentage_topics15k'] = percentage_topics15k

In [82]:
with open('/Users/cblanesg/Desktop/clean_speaches_topics.json', 'w') as f:
    json.dump(data.to_dict(), f)

In [83]:
data.to_excel('/Users/cblanesg/Desktop/clean_speaches_topics.xlsx')